In [ ]:
# Step 1: Import necessary libraries
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
# Step 2: Load and prepare the dataset
data = pd.read_csv("updating_multiclass_dataset.csv")  # Replace with your actual file path

# Display the first few rows to check the data structure
print(data.head())


# Map sentiments to integers (Adjust these mappings according to your dataset)
sentiment_mapping = {'positive': 2, 'negative': 0, 'neutral': 1}
data['Label'] = data['Sentiment'].map(sentiment_mapping)

# Check for NaN values in the 'Label' column
if data['Label'].isnull().any():
    print("NaN values found in labels. Please check your mapping.")
    print(data[data['Label'].isnull()])  # Print rows with NaN labels
else:
    print("All labels are mapped correctly.")

# Create a Hugging Face Dataset
dataset = Dataset.from_pandas(data)

In [ ]:
# Step 3: Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Step 4: Define the tokenization function
def tokenize_function(examples):
    # Tokenize the text and include the labels
    tokenized_output = tokenizer(examples['Text'], padding="max_length", truncation=True)
    tokenized_output['labels'] = examples['Label']  # Add labels to the tokenized output
    return tokenized_output

In [ ]:
# Step 5: Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
# Step 6: Check if labels are correctly added
print("Sample tokenized data:", tokenized_datasets[0])

In [ ]:
# Step 7: Split the dataset into training and validation sets
train_testvalid = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_testvalid['train']
small_train_dataset = train_dataset.shuffle(seed=42).select([i for i in list(range(1000))])  # Use 1000 samples
valid_dataset = train_testvalid['test']
small_valid_dataset = valid_dataset.shuffle(seed=42).select([i for i in list(range(200))])  # Use 200 samples

In [ ]:
# Step 8: Load the BERT model for sequence classification
num_labels = len(sentiment_mapping)  # Ensure num_labels corresponds to your label mapping
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="C:\\Users\\Aaditya Srivastava\\OneDrive\\Desktop\\BERT-model-main",  # Output directory
    eval_strategy="epoch",  # Evaluate only at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Try a higher batch size
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,  # Log every 100 steps for efficiency
    gradient_accumulation_steps=2,  # Accumulate gradients to simulate a larger batch
    fp16=True,  # Enable mixed precision training; disable if it slows down
    report_to="none", # Disable wandb logging
)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted') # Use weighted average for multiclass
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [ ]:
from transformers import DataCollatorWithPadding

# Step 8: Create a data collator
# Data collators are used to pad the input sequences to the same length
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Step 9: Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_valid_dataset,
    processing_class=tokenizer, # Use processing_class instead of tokenizer
    data_collator=data_collator,
    compute_metrics=compute_metrics # Add compute_metrics here
)

# Step 10: Start training
print("Starting training...")
trainer.train()
print("Training finished.")

In [ ]:
# Perform evaluation
metrics = trainer.evaluate()
print(metrics)

In [ ]:
# Step 12: Evaluate the model with additional metrics
print("Starting evaluation with more metrics...")
evaluation_results = trainer.evaluate(eval_dataset=small_valid_dataset)
print("Evaluation results:", evaluation_results)
print("Evaluation finished.")

## Hyperparameter tuning the model

In [ ]:
# Step 1: Modify TrainingArguments hyperparameters
training_args_tuned = TrainingArguments(
    output_dir="C:\\Users\\Aaditya Srivastava\\OneDrive\\Desktop\\BERT-model-main_tuned",
    eval_strategy="epoch",
    learning_rate=5e-5,  # Increased learning rate
    per_device_train_batch_size=16,  # Increased batch size
    per_device_eval_batch_size=16,
    num_train_epochs=3,  # Increased number of epochs
    weight_decay=0.01,
    logging_dir='./logs_tuned',
    logging_steps=100,
    gradient_accumulation_steps=1, # Reduced gradient accumulation
    fp16=True,
    report_to="none",
)

# Step 2: Re-initialize the Trainer with updated TrainingArguments
trainer_tuned = Trainer(
    model=model,
    args=training_args_tuned,
    train_dataset=small_train_dataset,
    eval_dataset=small_valid_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Step 3: Start training with the modified trainer
print("Starting tuned training...")
trainer_tuned.train()
print("Tuned training finished.")

# Step 4 & 5: Evaluate and print the results
print("Starting tuned evaluation...")
evaluation_results_tuned = trainer_tuned.evaluate(eval_dataset=small_valid_dataset)
print("Tuned evaluation results:", evaluation_results_tuned)
print("Tuned evaluation finished.")

In [ ]:
import os

# Create the directory
save_directory = "fine_tuned_bert_model"
os.makedirs(save_directory, exist_ok=True)

# Save the fine-tuned model
trainer_tuned.model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")

## Prediction/Inference

In [ ]:
# Step 13: Load the fine-tuned model and tokenizer
loaded_tokenizer = BertTokenizer.from_pretrained(save_directory)
loaded_model = BertForSequenceClassification.from_pretrained(save_directory)

# Step 14: Define a function for prediction
def predict_sentiment(text):
    # Tokenize the input text
    inputs = loaded_tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Make a prediction
    outputs = loaded_model(**inputs)
    logits = outputs.logits
    predictions = logits.argmax(dim=-1)

    # Map the prediction back to sentiment label
    # Create a reverse mapping from integer to sentiment label
    reverse_sentiment_mapping = {v: k for k, v in sentiment_mapping.items()}
    predicted_sentiment = reverse_sentiment_mapping[predictions.item()]

    return predicted_sentiment

# Step 15: Test the prediction function with some examples
example_texts = [
    "This is a fantastic movie!",
    "I am not happy with the service.",
    "The weather is neutral today.",
    "This is really bad.",
    "I love this product!"
]

print("Predictions:")
for text in example_texts:
    sentiment = predict_sentiment(text)
    print(f"Text: '{text}' -> Predicted Sentiment: {sentiment}")

In [ ]:
!pip install gradio -q

Now, let's create a simple interface using Gradio. This will create a web-based interface directly in your Colab output.

In [ ]:
import gradio as gr

# We'll use the predict_sentiment function you already defined
# from the previous section (Step 14).

# Create the Gradio interface
interface = gr.Interface(
    fn=predict_sentiment,  # Your prediction function
    inputs=gr.Textbox(lines=2, placeholder="Enter text here to analyze sentiment..."),  # Text input with improved placeholder
    outputs="text",  # Text output for sentiment
    title="BERT Sentiment Analysis", # Slightly modified title
    description="Enter text to analyze its sentiment (Positive, Negative, or Neutral) using a fine-tuned BERT model.", # More descriptive
    examples=[ # Add some examples
        ["I absolutely loved this movie, it was fantastic!"],
        ["The service was terrible, I'm very disappointed."],
        ["The weather is cloudy today with a chance of rain."],
        ["This product exceeded my expectations."],
        ["I have no strong feelings about this."]
    ]
)

# Launch the interface
print("Launching Gradio interface...")
interface.launch(share=True)